In [1]:
import pandas as pd
samples_path = '../logs/samples.csv'
samples = pd.read_csv(samples_path, index_col=0)
samples = [(s[0:25].tolist(), s[25:].tolist()) for s in samples.values][0:2]

In [2]:
from robustness import test_local_robustness

nnet_path = '../network/models/v3.2.0/model.nnet'
sym_results = test_local_robustness(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/robustness/symmetric')
asym_results = test_local_robustness(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, asym=True, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/robustness/asymmetric')

print('SYM RESULTS: ', sym_results)
print('ASYM RESULTS: ', asym_results)
# SYM RESULTS:  ((-0.00533, 0.00533), [(0.01606, 0.01606), (0.00533, 0.00533)])
# ASYM RESULTS:  ((-0.0072, 0.00606), [(0.04164, 0.02475), (0.0072, 0.00606)])

INFO:robustness:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:robustness:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:robustness:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:robustness:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:robustness:searching 9 coarse epsilons b/t 0.01 and 0.09
INFO:robustness:s0 coarse epsilon bounds: (0.01, 0.02) (4121ms)
INFO:robustness:searching 1000 epsilons b/t 0.01 and 0.01999
INFO:robustness:s0 epsilon: 0.01606 (2561ms)
INFO:robustness:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:robustness:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:robustness:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:robustness:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:robustness:s1 coarse epsilon bounds: (0.005, 0.006) (3175ms)
INFO:robustness:searching 100 epsilons b/t 0.005 and 0.00599
INFO:robustness:s1 epsilon: 0.00533 (834ms)
INFO:robustness:local robustness: (-0.00533, 0.00533) (10693ms)
INFO:robustness:w

In [3]:
from tot_net import TOTNet

def check_epsilons(net, sample, le, ue, asym=False):
    inputs, outputs = sample
    yval = max(outputs)
    y = outputs.index(yval)
    lbs = [x+le for x in inputs]
    ubs = [x+ue for x in inputs]
    result = None
    if asym:
        result = (net.find_counterexample(lbs, inputs, y),
                  net.find_counterexample(inputs, ubs, y))
    else:
        result = net.find_counterexample(lbs, ubs, y)
    y_duplicates = [oy for oy,oyval in enumerate(outputs) if oy != y and oyval == yval]
    if y_duplicates: print('WARNING: multiple y with same value', y_duplicates)
    return result

net = TOTNet(nnet_path)
sym_tests = []
asym_tests = []
sym_le, sym_ue = sym_results[0]
asym_le, asym_ue = asym_results[0]
for s,sample in enumerate(samples):
    sym_cexs = check_epsilons(net, sample, sym_le, sym_ue)
    sym_tests.append(sym_cexs)

    asym_cexs = check_epsilons(net, sample, asym_le, asym_ue, asym=True)
    asym_tests.append(asym_cexs)

print('Symmetric tests: ', sym_tests if any(sym_tests) else 'ok')
print('Asymmetric tests: ', asym_tests if any([l or u for l,u in asym_tests]) else 'ok')

Symmetric tests:  ok
Asymmetric tests:  ok


In [ ]:
# from maraboupy import Marabou, MarabouUtils
# 0:0.00231 1:0.01548

# (-0.005, 0.004)
# epsilon = 0.005
# sample = samples[0]
# inputs, outputs = sample
# net = TOTNet(nnet_path)
# # net.set_lower_bounds([x for x in inputs])
# net.set_upper_bounds([x for x in inputs])
# net.set_lower_bounds([x-epsilon for x in inputs])
# # net.set_upper_bounds([x+epsilon for x in inputs])
# net.set_expected_category(0)
# print(outputs)
# net.solve()

# net = Marabou.read_nnet(nnet_path)
# eq = Marabou.Equation(Marabou.Equation.LE)
# eq.addAddend(1, net.outputVars[0][1])
# eq.addAddend(-1, net.outputVars[0][0])
# eq.setScalar(0)
# net.addEquation(eq)
# MarabouUtils.addInequality(net, [net.outputVars[0][0], net.outputVars[0][1]], [1, -1], 0)
# net.solve()
# 0:0.00231 1:0.01548

# (-0.005, 0.004)
# epsilon = 0.005
# sample = samples[0]
# inputs, outputs = sample
# net = TOTNet(nnet_path)
# # net.set_lower_bounds([x for x in inputs])
# net.set_upper_bounds([x for x in inputs])
# net.set_lower_bounds([x-epsilon for x in inputs])
# # net.set_upper_bounds([x+epsilon for x in inputs])
# net.set_expected_category(0)
# print(outputs)
# net.solve()

# net = Marabou.read_nnet(nnet_path)
# eq = Marabou.Equation(Marabou.Equation.LE)
# eq.addAddend(1, net.outputVars[0][1])
# eq.addAddend(-1, net.outputVars[0][0])
# eq.setScalar(0)
# net.addEquation(eq)
# MarabouUtils.addInequality(net, [net.outputVars[0][0], net.outputVars[0][1]], [1, -1], 0)
# net.solve()